# AML CLI v2 と YAML によるモデルのデプロイ

[00_LoadData.ipynb](./00_LoadData.ipynb) を実行して、あらかじめデータセットの登録をおこなってください。

> Note : データセットのアップロード・登録を CLI v2 と YAML で記述できます。

## Azure へのログイン

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

## AML ワークスペースの環境設定

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## Managed Endpoint の作成

In [2]:
%%writefile managed_endpoint_config.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: my-endpoint
auth_mode: key

Writing managed_endpoint_config.yml


In [3]:
!az ml online-endpoint create --file managed_endpoint_config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "auth_mode": "key",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/onlineEndpoints/my-endpoint",
  "identity": {
    "principal_id": "b017f29d-fec6-4ff4-90fe-4e6fda44c66a",
    "tenant_id": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "type": "system_assigned"
  },
  "location": "eastus",
  "name": "my-endpoint",
  "properties": {
    "AzureAsyncOperationUri": "https://management.azure.com/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:cddd2b14-fb19-4ba4-bcab-61a9f6e4087a:005479de-bfb6-4e0b-a6dd-2710796957b7?api-version=2021-10-01",
    "azureml.onlineendpointid": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/aml-rg/providers/microsoft.machinelearnings

## Deployment 作成とトラフィックの割り当て

In [7]:
%%writefile deployment_config.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: deployment-v1
endpoint_name: my-endpoint
model: azureml:arima_model:1
code_configuration:
  code: scripts
  scoring_script: score.py
environment: 
  conda_file: scripts/arima-env.yml
  image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
instance_type: Standard_DS2_v2
instance_count: 1

Writing deployment_config.yml


In [10]:
!az ml online-deployment create --file deployment_config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --all-traffic

Command group 'ml online-deployment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Check: endpoint my-endpoint exists
Uploading scripts (0.01 MBs): 100%|█████| 5500/5500 [00:00<00:00, 145012.11it/s]


Creating/updating online deployment deployment-v1 ..............................................................................Done (6m 55s)
{
  "app_insights_enabled": false,
  "code_configuration": {
    "code": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/codes/c280d1ba-c0e9-4456-9037-beed355eb165/versions/1",
    "scoring_script": "score.py"
  },
  "endpoint_name": "my-endpoint",
  "environment": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/environments/CliV2AnonymousEnvironment/versions/100ed126-eca3-4573-9fbd-1bcc1332b444",
  "environment_variables